<a href="https://colab.research.google.com/github/HPSarma/University_Enrollment_Prediction/blob/main/UniversityEnrollmentPanelModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages('readxl')
install.packages('dplyr')
install.packages('FNN')
install.packages('caret')
install.packages('class')
install.packages('rpart.plot')
install.packages('rpart')
install.packages('glmnet')
install.packages('NeuralNetTools')
install.packages('PRROC')
install.packages("ROCR") # top plot lift curve
install.packages("tidyverse")
install.packages("skimr")
install.packages("mlbench")
install.packages("caret")
install.packages("FeatureHashing")
install.packages("glmnet")
install.packages("ivreg")
install.packages("mctest")
install.packages("readxl")
install.packages("forecast") #For the auto.arima() function
install.packages("ggplot2")  #For visualization
install.packages("Metrics")

install.packages("ROCR") # top plot lift curve
install.packages("tidyverse")
install.packages("skimr")


install.packages("ivreg")

install.packages("mltools")
install.packages("plm")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘future.apply’, ‘progressr’, ‘numDeriv’, ‘SQUAREM’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘Rcpp’, ‘gower’, ‘hardhat’, ‘ipred’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘shape

In [ ]:
# load the required libraries
library("readxl") # used to read excel files
library("dplyr") # used for data munging 
library("FNN") # used for knn regression (knn.reg function)
library("caret") # used for various predictive models
library("class") # for using confusion matrix function
library("rpart.plot") # used to plot decision tree
library("rpart")  # used for Regression tree
library("glmnet") # used for Lasso and Ridge regression
library('NeuralNetTools') # used to plot Neural Networks
library("PRROC") # top plot ROC curve
library("ROCR") # top plot lift curve
library("tidyverse")
library("skimr")
library("mlbench")
library("caret")
library("FeatureHashing")
library("glmnet")
#library("ivreg")
library("mctest")
library("readxl")
library("forecast") #For the auto.arima() function
library("ggplot2")  #For visualization
library("Metrics")  #For calculating the root mean square error
library(mltools)
library(data.table)
library("plm")


In [ ]:
library("ivreg")

Data loading and transformation



In [ ]:
fPanelname <- file.path("IPEDS Dataset 2007_2020_Formatted.csv") 
UPanelData <- read.csv(fPanelname,stringsAsFactors = FALSE)


In [ ]:
skim(UPanelData)

── Data Summary ────────────────────────
                           Values    
Name                       UPanelData
Number of rows             97859     
Number of columns          57        
_______________________              
Column type frequency:               
  character                5         
  numeric                  52        
________________________             
Group variables            None      

── Variable type: character ────────────────────────────────────────────────────
  skim_variable n_missing complete_rate min max empty n_unique whitespace
1 ADDR                  0             1   0  81   386    12789          0
2 CITY                  0             1   3  28     0     2896          0
3 INSTNM                0             1   3 105     0    12056          0
4 STABBR                0             1   2   2     0       59          0
5 ZIP                   0             1   3  10     0     9005          0

── Variable type: numeric ──────────────────────────

**Select the Data with relevant features**


In [ ]:
UPanelData <- UPanelData %>% select(-c("Year1", "FTEGD","FTEUG","ADDR","CITY","STABBR") )



In [ ]:
skim(UPanelData)

── Data Summary ────────────────────────
                           Values    
Name                       UPanelData
Number of rows             97859     
Number of columns          51        
_______________________              
Column type frequency:               
  character                2         
  numeric                  49        
________________________             
Group variables            None      

── Variable type: character ────────────────────────────────────────────────────
  skim_variable n_missing complete_rate min max empty n_unique whitespace
1 INSTNM                0             1   3 105     0    12056          0
2 ZIP                   0             1   3  10     0     9005          0

── Variable type: numeric ──────────────────────────────────────────────────────
   skim_variable n_missing complete_rate       mean         sd     p0      p25
 1 UNITID                0        1      276660.    131375.    100654 167835  
 2 Year                  0        1

In [ ]:
# # Change colname of one column
# #colnames(data)[colnames(data) == "Old_Name"] <- "New_Name"
 
# # Change colnames of all columns
# #colnames(data) <- c("New_Name1", "New_Name2", "New_Name3")
 
# # Change colnames of some columns
# colnames(UPanelData)[colnames(UPanelData) %in% c("BOARDAMT","CALSYS", "LEVEL1","LEVEL2","LEVEL3","LEVEL4","LEVEL5","LEVEL6","LEVEL7","LEVEL8")] <- c("BoardAmt","Calendar", "OneYearCert", "TwoYearCert","AssociateDegree","FourYearCert","Bachelor","PBCert","Masters","PMCert")
# colnames(UPanelData)[colnames(UPanelData) %in% c("RELAFFIL", "RMBRDAMT","ROOM","ROOMAMT","ROOMCAP","CARNEGIE","HLOFFER","INSTSIZE")] <- c("RelgAffil", "RoomBoardAmt", "RoomProv","RoomAmt","DormCapacity","CarnegieClass","HighOfferLevel","InstSize")
# colnames(UPanelData)[colnames(UPanelData) %in% c("LOCALE", "SECTOR","CINDFAM","CINDOFF","CINDON","CINSFAM","CINSOFF","CINSON","COTSFAM")] <- c("Location", "Sector", "PriceOffCampFam","PriceOffCamp","PriceInCamp","PriceInCampFam","PriceOffCampInState","PriceOffCampInStateFam","PriceOffCampOutStateFam")
# colnames(UPanelData)[colnames(UPanelData) %in% c("COTSOFF", "COTSON","TUITION1","TUITION2","TUITION3","TUITION5","TUITION6","TUITION7")] <- c("PriceOffCampOutState", "PriceInCampOutState", "TutionInDistrictUG","TutionInStateUG","TutionOutStateUG","TutionInDistrictG","TutionInStateG","TutionOutStateG")
# colnames(UPanelData)[colnames(UPanelData) %in% c("GRRTAN","GRRTAP","GRRTBK","GRRTHS","GRRTM","GRRTNR")] <- c("GradRateNat", "GradRatePacific","GradRateBlack","GradRateHispanic","GradRateMen","GradRateAlien")
# colnames(UPanelData)[colnames(UPanelData) %in% c("GRRTTOT","GRRTUN","GRRTW","GRRTWH","SALTOTL")] <- c("GradRate", "GradRateOther", "GradRateWomen","GradRateWhite","Avg9MonSalaryStaff")


In [ ]:
head(UPanelData,10)

,UNITID,Year,TOTENROLL,CARNEGIE,HLOFFER,INSTNM,INSTSIZE,LOCALE,SECTOR,ZIP,⋯,LEVEL4,LEVEL5,LEVEL6,LEVEL7,LEVEL8,RELAFFIL,RMBRDAMT,ROOM,ROOMAMT,ROOMCAP
,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,<chr>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,100654,2007,5742,16,9,Alabama A & M University,3,12,1,35762,⋯,0,1,0,1,1,-2,NA,1,2756,2843
2,100663,2007,14138,15,9,University of Alabama at Birmingham,4,12,1,35294-0110,⋯,1,1,1,1,1,-2,NA,1,4030,2150
3,100690,2007,519,51,9,Amridge University,1,12,2,36117-3553,⋯,0,1,0,1,0,74,NA,2,NA,NA
4,100706,2007,5754,16,9,University of Alabama in Huntsville,3,12,1,35899,⋯,0,1,1,1,1,-2,NA,1,4340,1330
5,100724,2007,5113,21,9,Alabama State University,3,12,1,36101-0271,⋯,0,1,0,1,1,-2,NA,1,2180,2083
6,100751,2007,22385,15,9,The University of Alabama,5,12,1,35487-0166,⋯,0,1,0,1,1,-2,NA,1,3750,6500
7,100760,2007,1918,40,3,Central Alabama Community College,2,41,4,35010,⋯,0,0,0,0,0,-2,NA,2,NA,NA
8,100812,2007,2339,32,5,Athens State University,2,31,1,35611,⋯,0,1,0,0,0,-2,NA,2,NA,NA
9,100830,2007,4153,21,8,Auburn University-Montgomery,3,12,1,36117-3596,⋯,0,1,0,1,1,-2,NA,1,3200,673


Remove columns with more than 15% null values - bad to leverage statistically

In [ ]:
na_threshold = 0.80


UPanelData = UPanelData[ ,which(colMeans(!is.na(UPanelData)) > na_threshold)]

In [ ]:
head(UPanelData,10)

,UNITID,Year,TOTENROLL,CARNEGIE,HLOFFER,INSTNM,INSTSIZE,LOCALE,SECTOR,ZIP,⋯,LEVEL1,LEVEL2,LEVEL3,LEVEL4,LEVEL5,LEVEL6,LEVEL7,LEVEL8,RELAFFIL,ROOM
,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,<chr>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,100654,2007,5742,16,9,Alabama A & M University,3,12,1,35762,⋯,0,0,0,0,1,0,1,1,-2,1
2,100663,2007,14138,15,9,University of Alabama at Birmingham,4,12,1,35294-0110,⋯,0,1,0,1,1,1,1,1,-2,1
3,100690,2007,519,51,9,Amridge University,1,12,2,36117-3553,⋯,0,0,1,0,1,0,1,0,74,2
4,100706,2007,5754,16,9,University of Alabama in Huntsville,3,12,1,35899,⋯,0,0,0,0,1,1,1,1,-2,1
5,100724,2007,5113,21,9,Alabama State University,3,12,1,36101-0271,⋯,0,0,0,0,1,0,1,1,-2,1
6,100751,2007,22385,15,9,The University of Alabama,5,12,1,35487-0166,⋯,0,0,0,0,1,0,1,1,-2,1
7,100760,2007,1918,40,3,Central Alabama Community College,2,41,4,35010,⋯,1,1,1,0,0,0,0,0,-2,2
8,100812,2007,2339,32,5,Athens State University,2,31,1,35611,⋯,0,0,0,0,1,0,0,0,-2,2
9,100830,2007,4153,21,8,Auburn University-Montgomery,3,12,1,36117-3596,⋯,0,0,0,0,1,0,1,1,-2,1


In [ ]:
names(UPanelData)

[1] "UNITID"    "Year"      "TOTENROLL" "CARNEGIE"  "HLOFFER"   "INSTNM"   
 [7] "INSTSIZE"  "LOCALE"    "SECTOR"    "ZIP"       "GRRTTOT"   "CALSYS"   
[13] "LEVEL1"    "LEVEL2"    "LEVEL3"    "LEVEL4"    "LEVEL5"    "LEVEL6"   
[19] "LEVEL7"    "LEVEL8"    "RELAFFIL"  "ROOM"

In [ ]:

 colnames(UPanelData) <- c("Id", "Year", "TOTENROLL", "CarnegieClass", "HighOfferLevel","InstName","InstSize","Location","Sector","Zip","GradRate","Calendar","OneYearCert","TwoYearCert","AssociateDegree","FourYearCert","Bachelor","PBCert","Masters","PMCert","RelgAffil","RoomProv")







In [ ]:
head(UPanelData,10)

,Id,Year,TOTENROLL,CarnegieClass,HighOfferLevel,InstName,InstSize,Location,Sector,Zip,⋯,OneYearCert,TwoYearCert,AssociateDegree,FourYearCert,Bachelor,PBCert,Masters,PMCert,RelgAffil,RoomProv
,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,<chr>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,100654,2007,5742,16,9,Alabama A & M University,3,12,1,35762,⋯,0,0,0,0,1,0,1,1,-2,1
2,100663,2007,14138,15,9,University of Alabama at Birmingham,4,12,1,35294-0110,⋯,0,1,0,1,1,1,1,1,-2,1
3,100690,2007,519,51,9,Amridge University,1,12,2,36117-3553,⋯,0,0,1,0,1,0,1,0,74,2
4,100706,2007,5754,16,9,University of Alabama in Huntsville,3,12,1,35899,⋯,0,0,0,0,1,1,1,1,-2,1
5,100724,2007,5113,21,9,Alabama State University,3,12,1,36101-0271,⋯,0,0,0,0,1,0,1,1,-2,1
6,100751,2007,22385,15,9,The University of Alabama,5,12,1,35487-0166,⋯,0,0,0,0,1,0,1,1,-2,1
7,100760,2007,1918,40,3,Central Alabama Community College,2,41,4,35010,⋯,1,1,1,0,0,0,0,0,-2,2
8,100812,2007,2339,32,5,Athens State University,2,31,1,35611,⋯,0,0,0,0,1,0,0,0,-2,2
9,100830,2007,4153,21,8,Auburn University-Montgomery,3,12,1,36117-3596,⋯,0,0,0,0,1,0,1,1,-2,1


In [ ]:
skim(UPanelData)

── Data Summary ────────────────────────
                           Values    
Name                       UPanelData
Number of rows             97859     
Number of columns          22        
_______________________              
Column type frequency:               
  character                2         
  numeric                  20        
________________________             
Group variables            None      

── Variable type: character ────────────────────────────────────────────────────
  skim_variable n_missing complete_rate min max empty n_unique whitespace
1 InstName              0             1   3 105     0    12056          0
2 Zip                   0             1   3  10     0     9005          0

── Variable type: numeric ──────────────────────────────────────────────────────
   skim_variable   n_missing complete_rate       mean         sd     p0    p25
 1 Id                      0         1     276660.    131375.    100654 167835
 2 Year                    0       

In [ ]:
#  # Change colnames of some columns
#  colnames(UPanelData)[colnames(UPanelData) %in% c("LEVEL1","CALSYS", "LEVEL2","LEVEL3","LEVEL4","LEVEL5","LEVEL6","LEVEL7","LEVEL8")] <- c("OneYearCert", "Calendar", "TwoYearCert","AssociateDegree","FourYearCert","Bachelor","PBCert","Masters","PMCert")
#  colnames(UPanelData)[colnames(UPanelData) %in% c("RELAFFIL","CARNEGIE","HLOFFER","INSTSIZE")] <- c("RelgAffil", "CarnegieClass","HighOfferLevel","InstSize")
#  colnames(UPanelData)[colnames(UPanelData) %in% c("LOCALE", "SECTOR","GRRTTOT","ROOM")] <- c("Location", "Sector", "GradRate","RoomProv")

In [ ]:
head(UPanelData,10)

,Id,Year,TOTENROLL,CarnegieClass,HighOfferLevel,InstName,InstSize,Location,Sector,Zip,⋯,OneYearCert,TwoYearCert,AssociateDegree,FourYearCert,Bachelor,PBCert,Masters,PMCert,RelgAffil,RoomProv
,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,<chr>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,100654,2007,5742,16,9,Alabama A & M University,3,12,1,35762,⋯,0,0,0,0,1,0,1,1,-2,1
2,100663,2007,14138,15,9,University of Alabama at Birmingham,4,12,1,35294-0110,⋯,0,1,0,1,1,1,1,1,-2,1
3,100690,2007,519,51,9,Amridge University,1,12,2,36117-3553,⋯,0,0,1,0,1,0,1,0,74,2
4,100706,2007,5754,16,9,University of Alabama in Huntsville,3,12,1,35899,⋯,0,0,0,0,1,1,1,1,-2,1
5,100724,2007,5113,21,9,Alabama State University,3,12,1,36101-0271,⋯,0,0,0,0,1,0,1,1,-2,1
6,100751,2007,22385,15,9,The University of Alabama,5,12,1,35487-0166,⋯,0,0,0,0,1,0,1,1,-2,1
7,100760,2007,1918,40,3,Central Alabama Community College,2,41,4,35010,⋯,1,1,1,0,0,0,0,0,-2,2
8,100812,2007,2339,32,5,Athens State University,2,31,1,35611,⋯,0,0,0,0,1,0,0,0,-2,2
9,100830,2007,4153,21,8,Auburn University-Montgomery,3,12,1,36117-3596,⋯,0,0,0,0,1,0,1,1,-2,1


In [ ]:
skim(UPanelData)

── Data Summary ────────────────────────
                           Values    
Name                       UPanelData
Number of rows             97859     
Number of columns          22        
_______________________              
Column type frequency:               
  character                2         
  numeric                  20        
________________________             
Group variables            None      

── Variable type: character ────────────────────────────────────────────────────
  skim_variable n_missing complete_rate min max empty n_unique whitespace
1 InstName              0             1   3 105     0    12056          0
2 Zip                   0             1   3  10     0     9005          0

── Variable type: numeric ──────────────────────────────────────────────────────
   skim_variable   n_missing complete_rate       mean         sd     p0    p25
 1 Id                      0         1     276660.    131375.    100654 167835
 2 Year                    0       

For Numeric features - GradRate and Total Staff replace with median values


In [ ]:
UPanelData <- UPanelData %>%
  mutate(GradRate  = ifelse(is.na(GradRate), 
                             median(GradRate, na.rm = TRUE), 
                             GradRate))
skim(UPanelData$GradRate) # NA = 0

# UPanelData <- UPanelData %>%
#   mutate(TotalStaff = ifelse(is.na(TotalStaff), 
#                              median(TotalStaff, na.rm = TRUE), 
#                              TotalStaff))
# skim(UPanelData$TotalStaff) # NA = 0

── Data Summary ────────────────────────
                           Values             
Name                       UPanelData$GradRate
Number of rows             97859              
Number of columns          1                  
_______________________                       
Column type frequency:                        
  numeric                  1                  
________________________                      
Group variables            None               

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate mean   sd p0 p25 p50 p75 p100 hist 
1 data                  0             1 53.6 23.0  0  37  55  70  100 ▂▅▇▆▃


In [ ]:
UPanelData <- UPanelData %>%
  mutate(Calendar = ifelse(is.na(Calendar), 
                             median(Calendar, na.rm = TRUE), 
                             Calendar))
skim(UPanelData$Calendar) # NA = 0

── Data Summary ────────────────────────
                           Values             
Name                       UPanelData$Calendar
Number of rows             97859              
Number of columns          1                  
_______________________                       
Column type frequency:                        
  numeric                  1                  
________________________                      
Group variables            None               

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate mean   sd p0 p25 p50 p75 p100 hist 
1 data                  0             1 3.39 2.62  1   1   2   6    7 ▇▁▁▁▅


In [ ]:
UPanelData <- UPanelData %>%
  mutate(OneYearCert = ifelse(is.na(OneYearCert), 
                             median(OneYearCert, na.rm = TRUE), 
                             OneYearCert))
skim(UPanelData$OneYearCert) # NA = 0

── Data Summary ────────────────────────
                           Values                
Name                       UPanelData$OneYearCert
Number of rows             97859                 
Number of columns          1                     
_______________________                          
Column type frequency:                           
  numeric                  1                     
________________________                         
Group variables            None                  

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate  mean    sd p0 p25 p50 p75 p100 hist 
1 data                  0             1 0.484 0.500 -1   0   0   1    1 ▁▁▇▁▇


In [ ]:
UPanelData <- UPanelData %>%
  mutate(TwoYearCert = ifelse(is.na(TwoYearCert), 
                             median(TwoYearCert, na.rm = TRUE), 
                             TwoYearCert))
skim(UPanelData$TwoYearCert) # NA = 0

── Data Summary ────────────────────────
                           Values                
Name                       UPanelData$TwoYearCert
Number of rows             97859                 
Number of columns          1                     
_______________________                          
Column type frequency:                           
  numeric                  1                     
________________________                         
Group variables            None                  

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate  mean    sd p0 p25 p50 p75 p100 hist 
1 data                  0             1 0.601 0.490 -1   0   1   1    1 ▁▁▅▁▇


In [ ]:
UPanelData <- UPanelData %>%
  mutate(AssociateDegree = ifelse(is.na(AssociateDegree), 
                             median(AssociateDegree, na.rm = TRUE), 
                             AssociateDegree))
skim(UPanelData$AssociateDegree) # NA = 0

── Data Summary ────────────────────────
                           Values                      
Name                       UPanelData$AssociateDegre...
Number of rows             97859                       
Number of columns          1                           
_______________________                                
Column type frequency:                                 
  numeric                  1                           
________________________                               
Group variables            None                        

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate  mean    sd p0 p25 p50 p75 p100 hist 
1 data                  0             1 0.440 0.496 -1   0   0   1    1 ▁▁▇▁▆


In [ ]:
UPanelData <- UPanelData %>%
  mutate(FourYearCert = ifelse(is.na(FourYearCert), 
                             median(FourYearCert, na.rm = TRUE), 
                             FourYearCert))
skim(UPanelData$FourYearCert) # NA = 0

── Data Summary ────────────────────────
                           Values                 
Name                       UPanelData$FourYearCert
Number of rows             97859                  
Number of columns          1                      
_______________________                           
Column type frequency:                            
  numeric                  1                      
________________________                          
Group variables            None                   

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate  mean    sd p0 p25 p50 p75 p100 hist 
1 data                  0             1 0.135 0.342 -1   0   0   0    1 ▁▁▇▁▁


In [ ]:
UPanelData <- UPanelData %>%
  mutate(Bachelor = ifelse(is.na(Bachelor), 
                             median(Bachelor, na.rm = TRUE), 
                             Bachelor))
skim(UPanelData$Bachelor) # NA = 0

── Data Summary ────────────────────────
                           Values             
Name                       UPanelData$Bachelor
Number of rows             97859              
Number of columns          1                  
_______________________                       
Column type frequency:                        
  numeric                  1                  
________________________                      
Group variables            None               

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate   mean   sd p0 p25 p50 p75 p100 hist 
1 data                  0             1 -0.156 1.19 -2  -2   0   1    1 ▆▁▁▇▇


In [ ]:
UPanelData <- UPanelData %>%
  mutate(PBCert = ifelse(is.na(PBCert), 
                             median(PBCert, na.rm = TRUE), 
                             PBCert))
skim(UPanelData$PBCert) # NA = 0

── Data Summary ────────────────────────
                           Values           
Name                       UPanelData$PBCert
Number of rows             97859            
Number of columns          1                
_______________________                     
Column type frequency:                      
  numeric                  1                
________________________                    
Group variables            None             

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate   mean   sd p0 p25 p50 p75 p100 hist 
1 data                  0             1 -0.463 1.06 -2  -2   0   0    1 ▅▁▁▇▂


In [ ]:
UPanelData <- UPanelData %>%
  mutate(Masters = ifelse(is.na(Masters), 
                             median(Masters, na.rm = TRUE), 
                             Masters))
skim(UPanelData$Masters) # NA = 0

── Data Summary ────────────────────────
                           Values            
Name                       UPanelData$Masters
Number of rows             97859             
Number of columns          1                 
_______________________                      
Column type frequency:                       
  numeric                  1                 
________________________                     
Group variables            None              

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate   mean   sd p0 p25 p50 p75 p100 hist 
1 data                  0             1 -0.311 1.18 -2  -2   0   1    1 ▆▁▁▇▆


In [ ]:
UPanelData <- UPanelData %>%
  mutate(PMCert = ifelse(is.na(PMCert), 
                             median(PMCert, na.rm = TRUE), 
                             PMCert))
skim(UPanelData$PMCert) # NA = 0

── Data Summary ────────────────────────
                           Values           
Name                       UPanelData$PMCert
Number of rows             97859            
Number of columns          1                
_______________________                     
Column type frequency:                      
  numeric                  1                
________________________                    
Group variables            None             

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate   mean   sd p0 p25 p50 p75 p100 hist 
1 data                  0             1 -0.501 1.02 -2  -2   0   0    1 ▃▁▁▇▁


In [ ]:
UPanelData <- UPanelData %>%
  mutate(RoomProv = ifelse(is.na(RoomProv), 
                             median(RoomProv, na.rm = TRUE), 
                             RoomProv))
skim(UPanelData$RoomProv) # NA = 0

── Data Summary ────────────────────────
                           Values             
Name                       UPanelData$RoomProv
Number of rows             97859              
Number of columns          1                  
_______________________                       
Column type frequency:                        
  numeric                  1                  
________________________                      
Group variables            None               

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate mean    sd p0 p25 p50 p75 p100 hist 
1 data                  0             1 1.69 0.463 -1   1   2   2    2 ▁▁▁▃▇


In [ ]:
UPanelData <- UPanelData %>%
  mutate(RelgAffil = ifelse(is.na(RelgAffil), 
                             median(RelgAffil, na.rm = TRUE), 
                             RelgAffil))
skim(UPanelData$RelgAffil) # NA = 0

── Data Summary ────────────────────────
                           Values              
Name                       UPanelData$RelgAffil
Number of rows             97859               
Number of columns          1                   
_______________________                        
Column type frequency:                         
  numeric                  1                   
________________________                       
Group variables            None                

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate mean   sd p0 p25 p50 p75 p100 hist 
1 data                  0             1 5.60 20.8 -2  -2  -2  -2  108 ▇▁▁▁▁


In [ ]:
skim(UPanelData)

── Data Summary ────────────────────────
                           Values    
Name                       UPanelData
Number of rows             97859     
Number of columns          22        
_______________________              
Column type frequency:               
  character                2         
  numeric                  20        
________________________             
Group variables            None      

── Variable type: character ────────────────────────────────────────────────────
  skim_variable n_missing complete_rate min max empty n_unique whitespace
1 InstName              0             1   3 105     0    12056          0
2 Zip                   0             1   3  10     0     9005          0

── Variable type: numeric ──────────────────────────────────────────────────────
   skim_variable   n_missing complete_rate       mean         sd     p0    p25
 1 Id                      0             1 276660.    131375.    100654 167835
 2 Year                    0       

In [ ]:
cor(UPanelData)

ERROR: ignored

From the Correlation, we see that the Enrollment is correlated with Carnegie Classification, Highest level of education offerings, Instituion Size,

Associate Degree, Bachelors, PB Certification, Masters, Post Masters Certification, 
a slight relation with One year Certificate, Four year Certificate

In [ ]:
UPanelData %>%  keep(is.factor) %>%  summary()



< table of extent 0 x 0 >

In [ ]:
summary(UPanelData[,"TOTENROLL"])

skim(UPanelData$TOTENROLL)


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
      0     132     505    2411    2098  330828 

── Data Summary ────────────────────────
                           Values              
Name                       UPanelData$TOTENROLL
Number of rows             97859               
Number of columns          1                   
_______________________                        
Column type frequency:                         
  numeric                  1                   
________________________                       
Group variables            None                

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate  mean    sd p0 p25 p50  p75   p100 hist 
1 data                  0             1 2411. 5835.  0 132 505 2098 330828 ▇▁▁▁▁


In [ ]:
UPanelData$TOTENROLL <- ifelse(UPanelData$TOTENROLL==0, 1, UPanelData$TOTENROLL)

In [ ]:
summary(UPanelData[,"GradRate"])

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   0.00   37.00   55.00   53.58   70.00  100.00 

In [ ]:
skim(UPanelData$GradRate)

── Data Summary ────────────────────────
                           Values             
Name                       UPanelData$GradRate
Number of rows             97859              
Number of columns          1                  
_______________________                       
Column type frequency:                        
  numeric                  1                  
________________________                      
Group variables            None               

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate mean   sd p0 p25 p50 p75 p100 hist 
1 data                  0             1 53.6 23.0  0  37  55  70  100 ▂▅▇▆▃


In [ ]:
UPanelData$GradRate <- ifelse(UPanelData$GradRate==0, 1, UPanelData$GradRate) 

In [ ]:
summary(UPanelData[,"InstSize"])

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 -2.000   1.000   1.000   1.651   2.000   5.000 

In [ ]:
UPanelData[,"InstSize"]

[1]  3  4  1  3  3  5  2  2  3  5  2  2  1  1  2  2  2  1  3  2  3  2  1  1
   [25]  1  1  1  3  2  3  3  1  2  2  2  1  2  2  2  2  3  2  2  2  1  1  2  2
   [49]  3  2  4  2  1  1  1  2  5  2  1  2  4  3  2  1  1  1  1  1  1  2  1  1
   [73]  2  1  1  2  1  1  1  1  5  3  5  1  1  2  1  1  2  1  1  1  1  2  1  3
   [97]  2  4  4  3  1  1  1  1  3  5  1  1  3  2  2  5  2  2  1  4  1  3  5  1
  [121]  1  2  1  4  1  1  4  2  1  1  2  3  1  3  4  2  1  1  1  1  1  1  1  4
  [145]  2  2  4  3  2  1  1  1  2  4  1  1  2  1  1  1  2  1  2  2  3  2  2  1
  [169]  1  2  1  1  1  1  2  2  1  1  2  1  2  2  2  1  2  1  2  2  1  3  2  2
  [193]  1  1  1  2  2  2  1  3  4  1  1  1  1  3  4  1  1  1  1  1  2  1  5  1
  [217]  4  2  1  1  3  4  1  2  1  1  1  3  1  2  1  1  4  1  2  4  2  2  2  2
  [241]  2  4  2  3  3  4  5  4  4  5  5  4  5  5  5  5  5  5  5  5  4  5  2  5
  [265]  4  1  1  1  1  1  1  2  1  1  3  4  1  1  1  1  1  1  1  1  1  5  2  4
  [289]  4  3  1  2  1  1  4  5  2  2  1  1  2  3  1  1  1  1  2  2  1  2  1  3
  [313]  4  3  4  3  1  4  5 -1  1  1  3  1  3  4  2  1  5  1  5  1  1  1  1  1
  [337]  1  3  2  1  2  1  2  1  2  1  1  4  1  1  5  2  2  5  1  1  3  2  4  1
  [361]  2  4  1  4  1  3  1  1  1  1  1  1  1  2  1  3  1  3  1  1  1  1  3  2
  [385]  1  1  1  1  2  1  4  1  2  3  1  2  1  4  2  1  1  2  2  5  1  3  4  3
  [409]  4  4  2  4  1  3  3  2  3  1  1  1  1  1  1  1  1  2  2  3  1  1  1  1
  [433]  1  1  1  1  1  1  1  2  1  1  4  3  2  4  1  3  1  2  4  1  1  3  4  5
  [457]  2  4  1  2  3  1  1  1  1  1  1  1  1  1  5  1  1  1  1  1  1  1  1  1
  [481]  2  2  1  4  5  2  3  1  1  1  1  2  1  1  2  3  2  5  1  1  5  1  3  1
  [505]  1  2  2  2  1  1  5  1  2  3  1  1  1  5  5  1  1  1  5  4  1  2  4  1
  [529]  5  4  5  3  1  1  5  1  3  4  1  1  1  3  5  4  1  1  1  5  3  5  5  1
  [553]  1  2  4  1  3  4  1  1  2  2  2  3  4  2  4  3  1  1  1  2  1  5  1  1
  [577]  1  5  3  1  1  1  1  2  1  1  1  1  4  1  4  3  5  1  2  3  4  2  1  2
  [601]  2  2  1  1  3  1  2  2  3  1  1  1  5  3  1  5  2  2  1  2  3  2  2  1
  [625]  5  2  1  2  1  2  1  3  1  1  3  4  2  2  4  1  1  1  1  1  3  5  2  2
  [649]  1  1  1  2  4  2  1  4  1  3  3  4  1  1  1  3  1  1  1  1  2  2  1  2
  [673]  1  2  1  2  1  1  1  1  2  1  4  2  1  2  1  1  5  2  2  3  3  1  2  1
  [697]  1  1  3  1  2  1  1  1  3  3  2  1  2  1  1  1  2  1  3  2  1  1  2  2
  [721]  2  3  3  1  2  3  1  4  1  1  2  2  1  2  3  1  4  1  1  2  2  3  2  5
  [745]  2  1  2  3  4  3  1  3  1  2  5  4  4  1  1  1  5  1  2  1  1  2  2  1
  [769]  3  1  2  2  1  1  4  5  1  2  3  1  5  1  2  1  1  4  1  2  4  1  2  4
  [793]  5  5  2  1  1  1  1  2  3  1  5  2  2  2  2  5  2  1  5  1  1  3  1  1
  [817]  3  1  5  1  1  4  2  1  2  2  1  2  1  4  1  1  2  1  2  1  2  2  1  2
  [841]  3  2  1  1  1  2  5  4  2  1  1  1  1  1  2  1  2  4  1  5  3  1  2  2
  [865]  5  3  4  2  3  1  1  1  1  1  2  1  2  2  1  1  4  5  2  4  1  2  4  2
  [889]  1  3  2  5  2  1  2  2  1  1  1  4  1  1  1  3  1  1  1  1  1  1  5  2
  [913]  1  1  1  1  4  1  1  2  1  2  2  1  1  1  2  2  2  1  1  3  2  1  2  1
  [937]  2  1  2  2  3  2  2  1  2  2  2  2  2  1  2  2  3  1  2  3  2  2  2  1
  [961]  2  2  2  1  4  2  2  1  4  2  3  3  1  2  4  5  5  2  2  1  1  2  2  2
  [985]  1  5  1  2  2  2  3  3  3  1  2  3  2  2  1  2  3  2  2  1  1  1  2  1
 [1009]  2  2  3  2  2  2  2  1  2  2  1  2  1  1  2  1  2  4  2  1  1  4  1  1
 [1033]  2  2  5  1  3  1  2  3  2  3  2  1  1  1  1  1  2  1  1  4  1  1  4  4
 [1057]  1  2  1  1  1  2  2  1  4  3  1  1  1  1  2  1  2  2  1  4  1  3  1  1
 [1081]  1  3  1  1  1  1  1  1  1  1  1  2  1  1  2  2  3  1  4  3  3  2  4  3
 [1105]  3  4  4  2  1  1  2  1  1  5  4  1  5  2  4  3  2  1  1  1  1  1  3  1
 [1129]  2  1  1  1  2  1  2  2  5  2  5  2  4  2  2  3  2  5  2  1  2  3 -1  2
 [1153]  2  4  2  2  3  2  2  2  1  4  2  1  3  1  3  3  1  2  2  3  1  4  1  1
 [1177]  1  1  1  1  1  3  2  1  1  1  2  2  4  1  3  1  3  1  1  2  2  5  1  4
 [12

In [ ]:
UPanelData$InstSize <- ifelse(UPanelData$InstSize<=1, 1, UPanelData$InstSize)

In [ ]:
summary(UPanelData[,"InstSize"])

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.000   1.000   1.000   1.655   2.000   5.000 

In [ ]:
UPanelData[,"InstSize"]

[1] 3 4 1 3 3 5 2 2 3 5 2 2 1 1 2 2 2 1 3 2 3 2 1 1 1 1 1 3 2 3 3 1 2 2 2 1
   [37] 2 2 2 2 3 2 2 2 1 1 2 2 3 2 4 2 1 1 1 2 5 2 1 2 4 3 2 1 1 1 1 1 1 2 1 1
   [73] 2 1 1 2 1 1 1 1 5 3 5 1 1 2 1 1 2 1 1 1 1 2 1 3 2 4 4 3 1 1 1 1 3 5 1 1
  [109] 3 2 2 5 2 2 1 4 1 3 5 1 1 2 1 4 1 1 4 2 1 1 2 3 1 3 4 2 1 1 1 1 1 1 1 4
  [145] 2 2 4 3 2 1 1 1 2 4 1 1 2 1 1 1 2 1 2 2 3 2 2 1 1 2 1 1 1 1 2 2 1 1 2 1
  [181] 2 2 2 1 2 1 2 2 1 3 2 2 1 1 1 2 2 2 1 3 4 1 1 1 1 3 4 1 1 1 1 1 2 1 5 1
  [217] 4 2 1 1 3 4 1 2 1 1 1 3 1 2 1 1 4 1 2 4 2 2 2 2 2 4 2 3 3 4 5 4 4 5 5 4
  [253] 5 5 5 5 5 5 5 5 4 5 2 5 4 1 1 1 1 1 1 2 1 1 3 4 1 1 1 1 1 1 1 1 1 5 2 4
  [289] 4 3 1 2 1 1 4 5 2 2 1 1 2 3 1 1 1 1 2 2 1 2 1 3 4 3 4 3 1 4 5 1 1 1 3 1
  [325] 3 4 2 1 5 1 5 1 1 1 1 1 1 3 2 1 2 1 2 1 2 1 1 4 1 1 5 2 2 5 1 1 3 2 4 1
  [361] 2 4 1 4 1 3 1 1 1 1 1 1 1 2 1 3 1 3 1 1 1 1 3 2 1 1 1 1 2 1 4 1 2 3 1 2
  [397] 1 4 2 1 1 2 2 5 1 3 4 3 4 4 2 4 1 3 3 2 3 1 1 1 1 1 1 1 1 2 2 3 1 1 1 1
  [433] 1 1 1 1 1 1 1 2 1 1 4 3 2 4 1 3 1 2 4 1 1 3 4 5 2 4 1 2 3 1 1 1 1 1 1 1
  [469] 1 1 5 1 1 1 1 1 1 1 1 1 2 2 1 4 5 2 3 1 1 1 1 2 1 1 2 3 2 5 1 1 5 1 3 1
  [505] 1 2 2 2 1 1 5 1 2 3 1 1 1 5 5 1 1 1 5 4 1 2 4 1 5 4 5 3 1 1 5 1 3 4 1 1
  [541] 1 3 5 4 1 1 1 5 3 5 5 1 1 2 4 1 3 4 1 1 2 2 2 3 4 2 4 3 1 1 1 2 1 5 1 1
  [577] 1 5 3 1 1 1 1 2 1 1 1 1 4 1 4 3 5 1 2 3 4 2 1 2 2 2 1 1 3 1 2 2 3 1 1 1
  [613] 5 3 1 5 2 2 1 2 3 2 2 1 5 2 1 2 1 2 1 3 1 1 3 4 2 2 4 1 1 1 1 1 3 5 2 2
  [649] 1 1 1 2 4 2 1 4 1 3 3 4 1 1 1 3 1 1 1 1 2 2 1 2 1 2 1 2 1 1 1 1 2 1 4 2
  [685] 1 2 1 1 5 2 2 3 3 1 2 1 1 1 3 1 2 1 1 1 3 3 2 1 2 1 1 1 2 1 3 2 1 1 2 2
  [721] 2 3 3 1 2 3 1 4 1 1 2 2 1 2 3 1 4 1 1 2 2 3 2 5 2 1 2 3 4 3 1 3 1 2 5 4
  [757] 4 1 1 1 5 1 2 1 1 2 2 1 3 1 2 2 1 1 4 5 1 2 3 1 5 1 2 1 1 4 1 2 4 1 2 4
  [793] 5 5 2 1 1 1 1 2 3 1 5 2 2 2 2 5 2 1 5 1 1 3 1 1 3 1 5 1 1 4 2 1 2 2 1 2
  [829] 1 4 1 1 2 1 2 1 2 2 1 2 3 2 1 1 1 2 5 4 2 1 1 1 1 1 2 1 2 4 1 5 3 1 2 2
  [865] 5 3 4 2 3 1 1 1 1 1 2 1 2 2 1 1 4 5 2 4 1 2 4 2 1 3 2 5 2 1 2 2 1 1 1 4
  [901] 1 1 1 3 1 1 1 1 1 1 5 2 1 1 1 1 4 1 1 2 1 2 2 1 1 1 2 2 2 1 1 3 2 1 2 1
  [937] 2 1 2 2 3 2 2 1 2 2 2 2 2 1 2 2 3 1 2 3 2 2 2 1 2 2 2 1 4 2 2 1 4 2 3 3
  [973] 1 2 4 5 5 2 2 1 1 2 2 2 1 5 1 2 2 2 3 3 3 1 2 3 2 2 1 2 3 2 2 1 1 1 2 1
 [1009] 2 2 3 2 2 2 2 1 2 2 1 2 1 1 2 1 2 4 2 1 1 4 1 1 2 2 5 1 3 1 2 3 2 3 2 1
 [1045] 1 1 1 1 2 1 1 4 1 1 4 4 1 2 1 1 1 2 2 1 4 3 1 1 1 1 2 1 2 2 1 4 1 3 1 1
 [1081] 1 3 1 1 1 1 1 1 1 1 1 2 1 1 2 2 3 1 4 3 3 2 4 3 3 4 4 2 1 1 2 1 1 5 4 1
 [1117] 5 2 4 3 2 1 1 1 1 1 3 1 2 1 1 1 2 1 2 2 5 2 5 2 4 2 2 3 2 5 2 1 2 3 1 2
 [1153] 2 4 2 2 3 2 2 2 1 4 2 1 3 1 3 3 1 2 2 3 1 4 1 1 1 1 1 1 1 3 2 1 1 1 2 2
 [1189] 4 1 3 1 3 1 1 2 2 5 1 4 4 4 1 2 3 1 3 1 2 1 2 2 2 2 3 1 2 3 2 2 1 2 1 3
 [1225] 2 2 1 2 1 2 1 1 2 1 1 1 2 1 1 1 5 4 1 1 3 1 2 2 4 1 1 3 1 4 2 4 1 2 1 4
 [1261] 2 2 2 1 1 1 1 2 1 1 2 1 2 1 2 1 1 2 1 1 1 1 1 1 2 2 4 2 3 2 3 3 2 2 3 2
 [1297] 2 2 4 5 1 2 1 1 1 1 1 2 2 3 4 2 3 5 2 3 2 1 1 1 1 2 1 1 1 1 1 1 2 2 2 4
 [1333] 1 1 1 1 1 1 1 4 2 1 1 3 2 1 1 2 1 2 2 2 2 1 1 1 1 2 2 2 1 4 1 1 1 2 2 1
 [1369] 1 1 1 2 2 2 2 1 4 1 2 3 2 1 1 3 1 1 2 2 2 1 1 1 3 1 2 3 2 1 1 1 5 1 3 5
 [1405] 4 1 1 1 1 2 2 1 2 1 2 2 4 2 4 2 2 2 2 1 2 1 2 1 1 1 2 2 2 1 1 1 1 3 1 1
 [1441] 2 1 1 3 2 1 2 1 2 2 2 1 1 1 1 1 3 1 1 1 2 2 2 1 2 1 2 1 3 1 3 1 2 2 2 1
 [1477] 1 1 2 2 2 4 1 1 3 5 2 1 5 1 1 1 2 1 1 1 1 2 2 1 1 1 1 1 3 2 1 1 2 1 1 2
 [1513] 1 1 3 1 4 1 1 2 2 2 1 2 2 1 2 1 2 4 2 1 1 1 1 2 1 4 3 1 1 2 1 2 1 2 2 1
 [1549] 4 1 1 2 1 2 1 5 1 1 1 2 1 5 1 2 2 2 2 3 1 4 2 4 1 1 3 1 2 2 1 1 1 1 1 1
 [1585] 1 3 2 2 2 2 2 2 2 1 2 4 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 1 1 1 1 4 1 1 1 1
 [1621] 1 2 1 1 1 3 1 1 1 1 1 1 1 1 1 2 2 5 2 2 1 1 1 1 2 1 4 2 1 3 1 4 2 3 1 3
 [1657] 1 1 1 3 1 2 2 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 2 1 1 2 1 4 3 2 2 4 2 1 1 1
 [1693] 1 4 1 1 1 2 1 2 1 1 2 1 2 1 1 2 2 2 2 2 1 3 2 2 2 4 1 2 1 2 1 1 1 2 1 3
 [1729] 4 1 1 1 1 1 2 4 1 1 1 1 3 3 1 3 1 2 3 2 2 2 1 1 3 2 1 2 1 2 3 2 3 1 4 1
 [1765] 2 3 1 5 3 4 5 2 2 1 1 5 3 2 1 1 2 4 2 1 1 1 3 2 2 1 1 4 1 1 2 2 1 2 1 2
 [18

In [ ]:
# UData$InstSize <- as.factor(UData$InstSize)

In [ ]:
#UData$TOTENROLL <- log(UData$TOTENROLL)

In [ ]:
#UData$TOTENROLL

In [ ]:
#summary(UData[,"TOTENROLL"])

#skim(UData$TOTENROLL)

Deal with Continuous features

Get the summary statistics for the continuous features.


In [ ]:
UPanelData %>%   keep(is.numeric) %>%   skim()



── Data Summary ────────────────────────
                           Values    
Name                       Piped data
Number of rows             97859     
Number of columns          20        
_______________________              
Column type frequency:               
  numeric                  20        
________________________             
Group variables            None      

── Variable type: numeric ──────────────────────────────────────────────────────
   skim_variable   n_missing complete_rate       mean         sd     p0    p25
 1 Id                      0             1 276660.    131375.    100654 167835
 2 Year                    0             1   2013.         3.93    2007   2010
 3 TOTENROLL               0             1   2411.      5835.         1    132
 4 CarnegieClass           0             1     16.1       21.0       -3     -3
 5 HighOfferLevel          0             1      4.58       2.55      -2      2
 6 InstSize                0             1      1.66       1.

In [ ]:
UPanelData$Calendar <- as.factor(UPanelData$Calendar)


In [ ]:
UPanelData$OneYearCert <- ifelse(UPanelData$OneYearCert>=1, 1, 0)

In [ ]:
UPanelData$OneYearCert <- as.factor(UPanelData$OneYearCert)

In [ ]:
UPanelData$TwoYearCert <- ifelse(UPanelData$TwoYearCert>=1, 1, 0)

In [ ]:
UPanelData$TwoYearCert <- as.factor(UPanelData$TwoYearCert)

In [ ]:
UPanelData$AssociateDegree <- ifelse(UPanelData$AssociateDegree>=1, 1, 0)

In [ ]:
UPanelData$AssociateDegree <- as.factor(UPanelData$AssociateDegree)

In [ ]:
UPanelData$FourYearCert <- ifelse(UPanelData$FourYearCert>=1, 1, 0)

In [ ]:
UPanelData$FourYearCert <- as.factor(UPanelData$FourYearCert)

In [ ]:
UPanelData$Bachelor <- ifelse(UPanelData$Bachelor>=1, 1, 0)

In [ ]:
UPanelData$Bachelor <- as.factor(UPanelData$Bachelor)

In [ ]:
UPanelData$PBCert <- ifelse(UPanelData$PBCert>=1, 1, 0)

In [ ]:
UPanelData$PBCert <- as.factor(UPanelData$PBCert)

In [ ]:
UPanelData$Masters <- ifelse(UPanelData$Masters>=1, 1, 0)

In [ ]:
UPanelData$Masters <- as.factor(UPanelData$Masters)

In [ ]:
UPanelData$PMCert <- ifelse(UPanelData$PMCert>=1, 1, 0)

In [ ]:
UPanelData$PMCert <- as.factor(UPanelData$PMCert)

In [ ]:
UPanelData$RelgAffil <- ifelse(UPanelData$RelgAffil>=1, 1, 0)

In [ ]:
UPanelData$RelgAffil <- as.factor(UPanelData$RelgAffil)

In [ ]:
UPanelData$RoomProv <- ifelse(UPanelData$RoomProv==1, 1, 0)

In [ ]:
UPanelData$RoomProv <- as.factor(UPanelData$RoomProv)

In [ ]:
UPanelData$CarnegieClass <- ifelse(UPanelData$CarnegieClass>=1, 1, 0)

In [ ]:
UPanelData$CarnegieClass <- as.factor(UPanelData$CarnegieClass)

In [ ]:
#UData$HighDegree <- ifelse(UData$HighDegree<=0, 1, UData$HighDegree)

In [ ]:
#UData$HighDegree <- as.factor(UData$HighDegree)

In [ ]:
UPanelData$HighOfferLevel <- ifelse(UPanelData$HighOfferLevel<=1, 1, UPanelData$HighOfferLevel)

In [ ]:
UPanelData$HighOfferLevel <- as.factor(UPanelData$HighOfferLevel)

In [ ]:
# UData$InstSize <- ifelse(UData$InstSize<=1, 1, UData$InstSize)

In [ ]:
# UData$InstSize <- as.factor(UData$InstSize)

In [ ]:
UPanelData$Location <- ifelse(UPanelData$Location<20, 1, ifelse(UPanelData$Location<30, 2, ifelse(UPanelData$Location<40, 3, 4)))

In [ ]:
UPanelData$Location <- as.factor(UPanelData$Location)

In [ ]:
UPanelData$Sector <- as.factor(UPanelData$Sector)

In [ ]:
UPanelData$InstSize <- as.factor(UPanelData$InstSize)

In [ ]:
head(UPanelData,10)

,Id,Year,TOTENROLL,CarnegieClass,HighOfferLevel,InstName,InstSize,Location,Sector,Zip,⋯,OneYearCert,TwoYearCert,AssociateDegree,FourYearCert,Bachelor,PBCert,Masters,PMCert,RelgAffil,RoomProv
,<int>,<int>,<dbl>,<fct>,<fct>,<chr>,<fct>,<fct>,<fct>,<chr>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,100654,2007,5742,1,9,Alabama A & M University,3,1,1,35762,⋯,0,0,0,0,1,0,1,1,0,1
2,100663,2007,14138,1,9,University of Alabama at Birmingham,4,1,1,35294-0110,⋯,0,1,0,1,1,1,1,1,0,1
3,100690,2007,519,1,9,Amridge University,1,1,2,36117-3553,⋯,0,0,1,0,1,0,1,0,1,0
4,100706,2007,5754,1,9,University of Alabama in Huntsville,3,1,1,35899,⋯,0,0,0,0,1,1,1,1,0,1
5,100724,2007,5113,1,9,Alabama State University,3,1,1,36101-0271,⋯,0,0,0,0,1,0,1,1,0,1
6,100751,2007,22385,1,9,The University of Alabama,5,1,1,35487-0166,⋯,0,0,0,0,1,0,1,1,0,1
7,100760,2007,1918,1,3,Central Alabama Community College,2,4,4,35010,⋯,1,1,1,0,0,0,0,0,0,0
8,100812,2007,2339,1,5,Athens State University,2,3,1,35611,⋯,0,0,0,0,1,0,0,0,0,0
9,100830,2007,4153,1,8,Auburn University-Montgomery,3,1,1,36117-3596,⋯,0,0,0,0,1,0,1,1,0,1


In [ ]:
skim(UPanelData)

── Data Summary ────────────────────────
                           Values    
Name                       UPanelData
Number of rows             97859     
Number of columns          22        
_______________________              
Column type frequency:               
  character                2         
  factor                   16        
  numeric                  4         
________________________             
Group variables            None      

── Variable type: character ────────────────────────────────────────────────────
  skim_variable n_missing complete_rate min max empty n_unique whitespace
1 InstName              0             1   3 105     0    12056          0
2 Zip                   0             1   3  10     0     9005          0

── Variable type: factor ───────────────────────────────────────────────────────
   skim_variable   n_missing complete_rate ordered n_unique
 1 CarnegieClass           0             1 FALSE          2
 2 HighOfferLevel          0       

In [ ]:
UPanelData <- one_hot(as.data.table(UPanelData))

In [ ]:
skim(UPanelData)

── Data Summary ────────────────────────
                           Values    
Name                       UPanelData
Number of rows             97859     
Number of columns          62        
Key                        NULL      
_______________________              
Column type frequency:               
  character                2         
  numeric                  60        
________________________             
Group variables            None      

── Variable type: character ────────────────────────────────────────────────────
  skim_variable n_missing complete_rate min max empty n_unique whitespace
1 InstName              0             1   3 105     0    12056          0
2 Zip                   0             1   3  10     0     9005          0

── Variable type: numeric ──────────────────────────────────────────────────────
   skim_variable     n_missing complete_rate         mean          sd     p0
 1 Id                        0             1 276660.      131375.     100654
 

In [ ]:
head(UPanelData,10)

Id,Year,TOTENROLL,CarnegieClass_0,CarnegieClass_1,HighOfferLevel_1,HighOfferLevel_2,HighOfferLevel_3,HighOfferLevel_4,HighOfferLevel_5,⋯,PBCert_0,PBCert_1,Masters_0,Masters_1,PMCert_0,PMCert_1,RelgAffil_0,RelgAffil_1,RoomProv_0,RoomProv_1
<int>,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
100654,2007,5742,0,1,0,0,0,0,0,⋯,1,0,0,1,0,1,1,0,0,1
100663,2007,14138,0,1,0,0,0,0,0,⋯,0,1,0,1,0,1,1,0,0,1
100690,2007,519,0,1,0,0,0,0,0,⋯,1,0,0,1,1,0,0,1,1,0
100706,2007,5754,0,1,0,0,0,0,0,⋯,0,1,0,1,0,1,1,0,0,1
100724,2007,5113,0,1,0,0,0,0,0,⋯,1,0,0,1,0,1,1,0,0,1
100751,2007,22385,0,1,0,0,0,0,0,⋯,1,0,0,1,0,1,1,0,0,1
100760,2007,1918,0,1,0,0,1,0,0,⋯,1,0,1,0,1,0,1,0,1,0
100812,2007,2339,0,1,0,0,0,0,1,⋯,1,0,1,0,1,0,1,0,1,0
100830,2007,4153,0,1,0,0,0,0,0,⋯,1,0,0,1,0,1,1,0,0,1


Panel Data Modeling - First Differences Estimation

In [ ]:
#write this sorted target list as a CSV file
write.csv(x=UPanelData, file = "UPanelData3.csv", row.names = TRUE)

In [ ]:
UPanelData101 <- plm.data(UPanelData, index=c("Id","Year"))

Warning message:
“use of 'plm.data' is discouraged, better use 'pdata.frame' instead”


In [ ]:
# plm(formula = log(as.data.frame(UPanelData$TOTENROLL)) ~ as.data.frame(lapply(UPanelData %>% select(-c("TOTENROLL")) , normalize)), data=UPanelData, model="fd")

In [ ]:
# plm(formula = log(UPanelData$TOTENROLL) ~ UPanelData$RelgAffil_0+UPanelData$RelgAffil_1+log(UPanelData$RoomBoardAmt)+UPanelData$RoomProv_0+UPanelData$RoomProv_1+
# UPanelData$Location_1+UPanelData$Location_2+UPanelData$Location_3+UPanelData$Location_4+UPanelData$Sector_1+UPanelData$Sector_2+UPanelData$Sector_3+UPanelData$Sector_4+
# UPanelData$Sector_5+UPanelData$Sector_6+UPanelData$Sector_7+UPanelData$Sector_8+UPanelData$Sector_9+UPanelData$GradRate+UPanelData$Calendar_1+UPanelData$Calendar_2+
# UPanelData$Calendar_3+UPanelData$Calendar_4+UPanelData$Calendar_5+UPanelData$Calendar_6+UPanelData$Calendar_7+UPanelData$OneYearCert_0+UPanelData$OneYearCert_1+
# UPanelData$TwoYearCert_0+UPanelData$TwoYearCert_1+UPanelData$AssociateDegree_0+UPanelData$AssociateDegree_1+UPanelData$FourYearCert_0+UPanelData$FourYearCert_1+
# UPanelData$Bachelor_0+UPanelData$Bachelor_1+UPanelData$PBCert_0+UPanelData$PBCert_1+UPanelData$Masters_0+UPanelData$Masters_1+UPanelData$PMCert_0+UPanelData$PMCert_1+
# log(UPanelData$RoomAmt)+UPanelData$CarnegieClass_0+UPanelData$CarnegieClass_1, data=UPanelData, model="fd")

Directly on the UData without converting to Panel

In [ ]:
plm(formula = log(UPanelData$TOTENROLL) ~ 
UPanelData$CarnegieClass_0
+UPanelData$CarnegieClass_1 
+HighOfferLevel_1
+HighOfferLevel_2
+HighOfferLevel_3
+HighOfferLevel_4
+HighOfferLevel_5
+HighOfferLevel_6
+HighOfferLevel_7
+HighOfferLevel_8
+HighOfferLevel_9
+InstSize_1
+InstSize_2
+InstSize_3
+InstSize_4
+InstSize_5
+UPanelData$Location_1
+UPanelData$Location_2
+UPanelData$Location_3
+UPanelData$Location_4
+UPanelData$Sector_1
+UPanelData$Sector_2
+UPanelData$Sector_3
+UPanelData$Sector_4
+UPanelData$Sector_5
+UPanelData$Sector_6
+UPanelData$Sector_7
+UPanelData$Sector_8
+UPanelData$Sector_9
+log(UPanelData$GradRate)
+UPanelData$Calendar_1
+UPanelData$Calendar_2
+UPanelData$Calendar_3
+UPanelData$Calendar_4
+UPanelData$Calendar_5
+UPanelData$Calendar_6
+UPanelData$Calendar_7
+UPanelData$OneYearCert_0
+UPanelData$OneYearCert_1
+UPanelData$TwoYearCert_0
+UPanelData$TwoYearCert_1
+UPanelData$AssociateDegree_0
+UPanelData$AssociateDegree_1
+UPanelData$FourYearCert_0
+UPanelData$FourYearCert_1
+UPanelData$Bachelor_0
+UPanelData$Bachelor_1
+UPanelData$PBCert_0
+UPanelData$PBCert_1
+UPanelData$Masters_0
+UPanelData$Masters_1
+UPanelData$PMCert_0
+UPanelData$PMCert_1
+UPanelData$RelgAffil_0
+UPanelData$RelgAffil_1
+UPanelData$RoomProv_0
+UPanelData$RoomProv_1,
data=UPanelData, 
model="fd")


Model Formula: log(UPanelData$TOTENROLL) ~ UPanelData$CarnegieClass_0 + UPanelData$CarnegieClass_1 + 
    HighOfferLevel_1 + HighOfferLevel_2 + HighOfferLevel_3 + 
    HighOfferLevel_4 + HighOfferLevel_5 + HighOfferLevel_6 + 
    HighOfferLevel_7 + HighOfferLevel_8 + HighOfferLevel_9 + 
    InstSize_1 + InstSize_2 + InstSize_3 + InstSize_4 + InstSize_5 + 
    UPanelData$Location_1 + UPanelData$Location_2 + UPanelData$Location_3 + 
    UPanelData$Location_4 + UPanelData$Sector_1 + UPanelData$Sector_2 + 
    UPanelData$Sector_3 + UPanelData$Sector_4 + UPanelData$Sector_5 + 
    UPanelData$Sector_6 + UPanelData$Sector_7 + UPanelData$Sector_8 + 
    UPanelData$Sector_9 + log(UPanelData$GradRate) + UPanelData$Calendar_1 + 
    UPanelData$Calendar_2 + UPanelData$Calendar_3 + UPanelData$Calendar_4 + 
    UPanelData$Calendar_5 + UPanelData$Calendar_6 + UPanelData$Calendar_7 + 
    UPanelData$OneYearCert_0 + UPanelData$OneYearCert_1 + UPanelData$TwoYearCert_0 + 
    UPanelData$TwoYearCert_1 + 

Using Panel Data - FE

In [ ]:
feModel = plm(formula = log(UPanelData101$TOTENROLL) ~ 
UPanelData101$CarnegieClass_0
+UPanelData101$CarnegieClass_1 
+HighOfferLevel_1
+HighOfferLevel_2
+HighOfferLevel_3
+HighOfferLevel_4
+HighOfferLevel_5
+HighOfferLevel_6
+HighOfferLevel_7
+HighOfferLevel_8
+HighOfferLevel_9
+InstSize_1
+InstSize_2
+InstSize_3
+InstSize_4
+InstSize_5
+UPanelData101$Location_1
+UPanelData101$Location_2
+UPanelData101$Location_3
+UPanelData101$Location_4
+UPanelData101$Sector_1
+UPanelData101$Sector_2
+UPanelData101$Sector_3
+UPanelData101$Sector_4
+UPanelData101$Sector_5
+UPanelData101$Sector_6
+UPanelData101$Sector_7
+UPanelData101$Sector_8
+UPanelData101$Sector_9
+log(UPanelData101$GradRate)
+UPanelData101$Calendar_1
+UPanelData101$Calendar_2
+UPanelData101$Calendar_3
+UPanelData101$Calendar_4
+UPanelData101$Calendar_5
+UPanelData101$Calendar_6
+UPanelData101$Calendar_7
+UPanelData101$OneYearCert_0
+UPanelData101$OneYearCert_1
+UPanelData101$TwoYearCert_0
+UPanelData101$TwoYearCert_1
+UPanelData101$AssociateDegree_0
+UPanelData101$AssociateDegree_1
+UPanelData101$FourYearCert_0
+UPanelData101$FourYearCert_1
+UPanelData101$Bachelor_0
+UPanelData101$Bachelor_1
+UPanelData101$PBCert_0
+UPanelData101$PBCert_1
+UPanelData101$Masters_0
+UPanelData101$Masters_1
+UPanelData101$PMCert_0
+UPanelData101$PMCert_1
+UPanelData101$RelgAffil_0
+UPanelData101$RelgAffil_1
+UPanelData101$RoomProv_0
+UPanelData101$RoomProv_1,
data=UPanelData101, 
model="fd")

In [ ]:
summary(feModel)

Oneway (individual) effect First-Difference Model

Call:
plm(formula = log(UPanelData101$TOTENROLL) ~ UPanelData101$CarnegieClass_0 + 
    UPanelData101$CarnegieClass_1 + HighOfferLevel_1 + HighOfferLevel_2 + 
    HighOfferLevel_3 + HighOfferLevel_4 + HighOfferLevel_5 + 
    HighOfferLevel_6 + HighOfferLevel_7 + HighOfferLevel_8 + 
    HighOfferLevel_9 + InstSize_1 + InstSize_2 + InstSize_3 + 
    InstSize_4 + InstSize_5 + UPanelData101$Location_1 + UPanelData101$Location_2 + 
    UPanelData101$Location_3 + UPanelData101$Location_4 + UPanelData101$Sector_1 + 
    UPanelData101$Sector_2 + UPanelData101$Sector_3 + UPanelData101$Sector_4 + 
    UPanelData101$Sector_5 + UPanelData101$Sector_6 + UPanelData101$Sector_7 + 
    UPanelData101$Sector_8 + UPanelData101$Sector_9 + log(UPanelData101$GradRate) + 
    UPanelData101$Calendar_1 + UPanelData101$Calendar_2 + UPanelData101$Calendar_3 + 
    UPanelData101$Calendar_4 + UPanelData101$Calendar_5 + UPanelData101$Calendar_6 + 
    UPanelData101

Within

In [ ]:
feWithin = plm(formula = log(UPanelData101$TOTENROLL) ~ 
UPanelData101$CarnegieClass_0
+UPanelData101$CarnegieClass_1 
+HighOfferLevel_1
+HighOfferLevel_2
+HighOfferLevel_3
+HighOfferLevel_4
+HighOfferLevel_5
+HighOfferLevel_6
+HighOfferLevel_7
+HighOfferLevel_8
+HighOfferLevel_9
+InstSize_1
+InstSize_2
+InstSize_3
+InstSize_4
+InstSize_5
+UPanelData101$Location_1
+UPanelData101$Location_2
+UPanelData101$Location_3
+UPanelData101$Location_4
+UPanelData101$Sector_1
+UPanelData101$Sector_2
+UPanelData101$Sector_3
+UPanelData101$Sector_4
+UPanelData101$Sector_5
+UPanelData101$Sector_6
+UPanelData101$Sector_7
+UPanelData101$Sector_8
+UPanelData101$Sector_9
+log(UPanelData101$GradRate)
+UPanelData101$Calendar_1
+UPanelData101$Calendar_2
+UPanelData101$Calendar_3
+UPanelData101$Calendar_4
+UPanelData101$Calendar_5
+UPanelData101$Calendar_6
+UPanelData101$Calendar_7
+UPanelData101$OneYearCert_0
+UPanelData101$OneYearCert_1
+UPanelData101$TwoYearCert_0
+UPanelData101$TwoYearCert_1
+UPanelData101$AssociateDegree_0
+UPanelData101$AssociateDegree_1
+UPanelData101$FourYearCert_0
+UPanelData101$FourYearCert_1
+UPanelData101$Bachelor_0
+UPanelData101$Bachelor_1
+UPanelData101$PBCert_0
+UPanelData101$PBCert_1
+UPanelData101$Masters_0
+UPanelData101$Masters_1
+UPanelData101$PMCert_0
+UPanelData101$PMCert_1
+UPanelData101$RelgAffil_0
+UPanelData101$RelgAffil_1
+UPanelData101$RoomProv_0
+UPanelData101$RoomProv_1,
data=UPanelData101, 
model="within")

In [ ]:
summary(feWithin)

Oneway (individual) effect Within Model

Call:
plm(formula = log(UPanelData101$TOTENROLL) ~ UPanelData101$CarnegieClass_0 + 
    UPanelData101$CarnegieClass_1 + HighOfferLevel_1 + HighOfferLevel_2 + 
    HighOfferLevel_3 + HighOfferLevel_4 + HighOfferLevel_5 + 
    HighOfferLevel_6 + HighOfferLevel_7 + HighOfferLevel_8 + 
    HighOfferLevel_9 + InstSize_1 + InstSize_2 + InstSize_3 + 
    InstSize_4 + InstSize_5 + UPanelData101$Location_1 + UPanelData101$Location_2 + 
    UPanelData101$Location_3 + UPanelData101$Location_4 + UPanelData101$Sector_1 + 
    UPanelData101$Sector_2 + UPanelData101$Sector_3 + UPanelData101$Sector_4 + 
    UPanelData101$Sector_5 + UPanelData101$Sector_6 + UPanelData101$Sector_7 + 
    UPanelData101$Sector_8 + UPanelData101$Sector_9 + log(UPanelData101$GradRate) + 
    UPanelData101$Calendar_1 + UPanelData101$Calendar_2 + UPanelData101$Calendar_3 + 
    UPanelData101$Calendar_4 + UPanelData101$Calendar_5 + UPanelData101$Calendar_6 + 
    UPanelData101$Calendar_

Random

In [ ]:
pool = plm(formula = log(UPanelData101$TOTENROLL) ~ 
UPanelData101$CarnegieClass_0
+UPanelData101$CarnegieClass_1 
+HighOfferLevel_1
+HighOfferLevel_2
+HighOfferLevel_3
+HighOfferLevel_4
+HighOfferLevel_5
+HighOfferLevel_6
+HighOfferLevel_7
+HighOfferLevel_8
+HighOfferLevel_9
+InstSize_1
+InstSize_2
+InstSize_3
+InstSize_4
+InstSize_5
+UPanelData101$Location_1
+UPanelData101$Location_2
+UPanelData101$Location_3
+UPanelData101$Location_4
+UPanelData101$Sector_1
+UPanelData101$Sector_2
+UPanelData101$Sector_3
+UPanelData101$Sector_4
+UPanelData101$Sector_5
+UPanelData101$Sector_6
+UPanelData101$Sector_7
+UPanelData101$Sector_8
+UPanelData101$Sector_9
+log(UPanelData101$GradRate)
+UPanelData101$Calendar_1
+UPanelData101$Calendar_2
+UPanelData101$Calendar_3
+UPanelData101$Calendar_4
+UPanelData101$Calendar_5
+UPanelData101$Calendar_6
+UPanelData101$Calendar_7
+UPanelData101$OneYearCert_0
+UPanelData101$OneYearCert_1
+UPanelData101$TwoYearCert_0
+UPanelData101$TwoYearCert_1
+UPanelData101$AssociateDegree_0
+UPanelData101$AssociateDegree_1
+UPanelData101$FourYearCert_0
+UPanelData101$FourYearCert_1
+UPanelData101$Bachelor_0
+UPanelData101$Bachelor_1
+UPanelData101$PBCert_0
+UPanelData101$PBCert_1
+UPanelData101$Masters_0
+UPanelData101$Masters_1
+UPanelData101$PMCert_0
+UPanelData101$PMCert_1
+UPanelData101$RelgAffil_0
+UPanelData101$RelgAffil_1
+UPanelData101$RoomProv_0
+UPanelData101$RoomProv_1,
data=UPanelData101, 
model="pooling")

In [ ]:
summary(pool)

Pooling Model

Call:
plm(formula = log(UPanelData101$TOTENROLL) ~ UPanelData101$CarnegieClass_0 + 
    UPanelData101$CarnegieClass_1 + HighOfferLevel_1 + HighOfferLevel_2 + 
    HighOfferLevel_3 + HighOfferLevel_4 + HighOfferLevel_5 + 
    HighOfferLevel_6 + HighOfferLevel_7 + HighOfferLevel_8 + 
    HighOfferLevel_9 + InstSize_1 + InstSize_2 + InstSize_3 + 
    InstSize_4 + InstSize_5 + UPanelData101$Location_1 + UPanelData101$Location_2 + 
    UPanelData101$Location_3 + UPanelData101$Location_4 + UPanelData101$Sector_1 + 
    UPanelData101$Sector_2 + UPanelData101$Sector_3 + UPanelData101$Sector_4 + 
    UPanelData101$Sector_5 + UPanelData101$Sector_6 + UPanelData101$Sector_7 + 
    UPanelData101$Sector_8 + UPanelData101$Sector_9 + log(UPanelData101$GradRate) + 
    UPanelData101$Calendar_1 + UPanelData101$Calendar_2 + UPanelData101$Calendar_3 + 
    UPanelData101$Calendar_4 + UPanelData101$Calendar_5 + UPanelData101$Calendar_6 + 
    UPanelData101$Calendar_7 + UPanelData101$OneYearC

OLS on the Panel

In [ ]:
fixed.dum <-lm(log(UPanelData101$TOTENROLL) ~ 
UPanelData101$CarnegieClass_0
+UPanelData101$CarnegieClass_1 
+HighOfferLevel_1
+HighOfferLevel_2
+HighOfferLevel_3
+HighOfferLevel_4
+HighOfferLevel_5
+HighOfferLevel_6
+HighOfferLevel_7
+HighOfferLevel_8
+HighOfferLevel_9
+InstSize_1
+InstSize_2
+InstSize_3
+InstSize_4
+InstSize_5
+UPanelData101$Location_1
+UPanelData101$Location_2
+UPanelData101$Location_3
+UPanelData101$Location_4
+UPanelData101$Sector_1
+UPanelData101$Sector_2
+UPanelData101$Sector_3
+UPanelData101$Sector_4
+UPanelData101$Sector_5
+UPanelData101$Sector_6
+UPanelData101$Sector_7
+UPanelData101$Sector_8
+UPanelData101$Sector_9
+log(UPanelData101$GradRate)
+UPanelData101$Calendar_1
+UPanelData101$Calendar_2
+UPanelData101$Calendar_3
+UPanelData101$Calendar_4
+UPanelData101$Calendar_5
+UPanelData101$Calendar_6
+UPanelData101$Calendar_7
+UPanelData101$OneYearCert_0
+UPanelData101$OneYearCert_1
+UPanelData101$TwoYearCert_0
+UPanelData101$TwoYearCert_1
+UPanelData101$AssociateDegree_0
+UPanelData101$AssociateDegree_1
+UPanelData101$FourYearCert_0
+UPanelData101$FourYearCert_1
+UPanelData101$Bachelor_0
+UPanelData101$Bachelor_1
+UPanelData101$PBCert_0
+UPanelData101$PBCert_1
+UPanelData101$Masters_0
+UPanelData101$Masters_1
+UPanelData101$PMCert_0
+UPanelData101$PMCert_1
+UPanelData101$RelgAffil_0
+UPanelData101$RelgAffil_1
+UPanelData101$RoomProv_0
+UPanelData101$RoomProv_1, data = UPanelData101)


In [ ]:
summary(fixed.dum)


Call:
lm(formula = log(UPanelData101$TOTENROLL) ~ UPanelData101$CarnegieClass_0 + 
    UPanelData101$CarnegieClass_1 + HighOfferLevel_1 + HighOfferLevel_2 + 
    HighOfferLevel_3 + HighOfferLevel_4 + HighOfferLevel_5 + 
    HighOfferLevel_6 + HighOfferLevel_7 + HighOfferLevel_8 + 
    HighOfferLevel_9 + InstSize_1 + InstSize_2 + InstSize_3 + 
    InstSize_4 + InstSize_5 + UPanelData101$Location_1 + UPanelData101$Location_2 + 
    UPanelData101$Location_3 + UPanelData101$Location_4 + UPanelData101$Sector_1 + 
    UPanelData101$Sector_2 + UPanelData101$Sector_3 + UPanelData101$Sector_4 + 
    UPanelData101$Sector_5 + UPanelData101$Sector_6 + UPanelData101$Sector_7 + 
    UPanelData101$Sector_8 + UPanelData101$Sector_9 + log(UPanelData101$GradRate) + 
    UPanelData101$Calendar_1 + UPanelData101$Calendar_2 + UPanelData101$Calendar_3 + 
    UPanelData101$Calendar_4 + UPanelData101$Calendar_5 + UPanelData101$Calendar_6 + 
    UPanelData101$Calendar_7 + UPanelData101$OneYearCert_0 + 
    UP

In [ ]:
plmtest(pool)
#feRandom


	Lagrange Multiplier Test - (Honda) for unbalanced panels

data:  log(UPanelData101$TOTENROLL) ~ UPanelData101$CarnegieClass_0 +  ...
normal = 425.43, p-value < 2.2e-16
alternative hypothesis: significant effects


The test statistic is 425.43 and the p-value is less than 0.05
it is significant and teh null hypothesis is in favor os OLS is rejected
Thus, the Random Effects model is chosen agains the OLS

Now, to chooe between FE and OLS
The LM test to choose between the fixed effects model and the OLS is done using the function PFtest on the fixed and pooled estimates

The null hypothesis is the there is no time-invariant effect and so OLS should be used

In [ ]:
pFtest(feWithin,feRandom)


	F test for individual effects

data:  log(UPanelData101$TOTENROLL) ~ UPanelData101$CarnegieClass_0 +  ...
F = 30.635, df1 = 9162, df2 = 88656, p-value < 2.2e-16
alternative hypothesis: significant effects


The test stat is 30.635 and p-value is less than 0.05
The null hypothesis is rejected thus impyling that the FE model should be used instead of OLS

In [ ]:
pFtest(feWithin, feModel)

Warning message in pf(stat, df1, df2, lower.tail = FALSE):
“NaNs produced”



	F test for individual effects

data:  log(UPanelData101$TOTENROLL) ~ UPanelData101$CarnegieClass_0 +  ...
F = 24848, df1 = -1, df2 = 88656, p-value = NA
alternative hypothesis: significant effects


Choose between FE and RE
Hausman test
Panel Hausman Ttes- phtest

In [ ]:
phtest(feRandom, feWithin)


	Hausman Test

data:  log(UPanelData101$TOTENROLL) ~ UPanelData101$CarnegieClass_0 +  ...
chisq = 119025, df = 40, p-value < 2.2e-16
alternative hypothesis: one model is inconsistent


Conclusion
- Panel data modeling is method to estimate data whic is both time-series and cross-sectional
- it accounts for individual-specific heterogeneity

##########################################################################################



<!-- **YET To RUN** -->
<!-- Recursive feature Elimination  #used for Classification problem -->`
<!-- ```{r echo=TRUE, warning=FALSE, message=FALSE} -->
<!-- # ensure the results are repeatable -->
<!-- set.seed(7) -->
<!-- # define the control using a random forest selection function -->
<!-- control <- rfeControl(functions=rfFuncs, method="cv", number=10) -->
<!-- # run the RFE algorithm -->
<!-- results <- rfe(PimaIndiansDiabetes[,2:53], PimaIndiansDiabetes[,1], sizes=c(1:52), rfeControl=control) -->
<!-- # summarize the results -->
<!-- print(results) -->
<!-- # list the chosen features -->
<!-- predictors(results) -->
<!-- # plot the results -->
<!-- plot(results, type=c("g", "o")) -->
<!-- ``` -->
<!-- **YET TO RUN** -->
<!-- **LASSO/ RIDGE/ELASTIC NET** -->

<!-- **YET TO RUN** -->

<!-- https://amunategui.github.io/feature-hashing/index.html -->

<!-- ```{r echo=TRUE, warning=FALSE, message=FALSE} -->
<!-- UData_featured <- UData %>% select(c("TOTENROLL","CALSYS","LEVEL1")) -->
<!-- targetEnrol <- UData_featured$TOTENROLL -->
<!-- ``` -->
<!-- **A. Using Dummy Variables** -->
<!-- ```{r echo=TRUE, warning=FALSE, message=FALSE} -->
<!-- UData_dummy <- UData_featured -->
<!-- require(caret) -->
<!-- dmy <- dummyVars(" ~ .", data = UData_dummy) -->
<!-- UData_dummy <- data.frame(predict(dmy, newdata = UData_dummy)) -->
<!-- dim(UData_dummy) -->
<!-- ``` -->
<!-- ```{r echo=TRUE, warning=FALSE, message=FALSE} -->
<!-- # split the data into training and testing data sets -->
<!-- set.seed(1234) -->
<!-- split <- sample(nrow(UData_dummy), floor(0.5*nrow(UData_dummy))) -->
<!-- objTrain <-UData_dummy[split,] -->
<!-- objTest <- UData_dummy[-split,] -->

<!-- predictorName <- setdiff(names(UData_dummy),targetEnrol) -->
<!-- ``` -->
<!-- ##Prediction - change to Linear regression -->
<!-- ```{r echo=TRUE, warning=FALSE, message=FALSE} -->
<!-- # straight matrix model not recommended - works but very slow, go with a sparse matrix -->
<!-- # glmnetModel <- cv.glmnet(model.matrix(~., data=objTrain[,predictorNames]), objTrain[,outcomeName], -->
<!-- #             family = "binomial", type.measure = "auc") -->

<!-- glmnetModel <- cv.glmnet(sparse.model.matrix(~., data=objTrain[,predictorName]), objTrain[,targetEnrol], -->
<!--                          family = "binomial", type.measure = "auc") -->
<!-- glmnetPredict <- predict(glmnetModel,sparse.model.matrix(~., data=objTest[,predictorName]), s="lambda.min") -->

<!-- ``` -->

<!-- Let’s look at the AUC (Area under the curve) so we can compare this approach with the feature-hashed one: -->
<!-- ```{r echo=TRUE, warning=FALSE, message=FALSE} -->
<!-- auc(objTest[,targetEnrol], glmnetPredict) -->
<!-- ## [1] 0.6480986 -->
<!-- ``` -->
<!-- **B. Using Feature Hashing** -->
<!-- ```{r echo=TRUE, warning=FALSE, message=FALSE} -->
<!-- UData_hashed <- UData_featured -->
<!-- predictorNames <- setdiff(names(UData_hashed),targetEnrol) -->
<!-- ``` -->
<!-- **SPlit** -->
<!-- ```{r echo=TRUE, warning=FALSE, message=FALSE} -->
<!-- set.seed(1234) -->
<!-- split <- sample(nrow(UData_hashed), floor(0.5*nrow(UData_hashed))) -->
<!-- objTrain <-UData_hashed[split,] -->
<!-- objTest <- UData_hashed[-split,] -->
<!-- ``` -->
<!-- ```{r echo=TRUE, warning=FALSE, message=FALSE} -->

<!-- objTrain_hashed = hashed.model.matrix(~., data=objTrain[,predictorNames], hash.size=2^4, transpose=FALSE) -->
<!-- objTrain_hashed = as(objTrain_hashed, "dgCMatrix") -->
<!-- objTest_hashed = hashed.model.matrix(~., data=objTest[,predictorNames], hash.size=2^4, transpose=FALSE) -->
<!-- objTest_hashed = as(objTest_hashed, "dgCMatrix") -->
<!-- ``` -->
<!-- ##Prediction - change to Linear regression -->
<!-- ```{r echo=TRUE, warning=FALSE, message=FALSE} -->
<!-- glmnetModel <- cv.glmnet(objTrain_hashed, objTrain[,targetEnrol], -->
<!--                      family = "binomial", type.measure = "auc") -->
<!-- ``` -->

<!-- Let’s see how this version scored: -->
<!-- ```{r echo=TRUE, warning=FALSE, message=FALSE} -->
<!-- glmnetPredict <- predict(glmnetModel, objTest_hashed, s="lambda.min") -->
<!-- auc(objTest[,targetEnrol], glmnetPredict) -->
<!-- ## [1] 0.6475847 -->
<!-- ``` -->


In [ ]:
CarnegieClass_2017 
+HighOfferLevel_1_2017
+HighOfferLevel_2_2017
+HighOfferLevel_3_2017
+HighOfferLevel_4_2017
+HighOfferLevel_5_2017
+HighOfferLevel_6_2017
+HighOfferLevel_7_2017
+HighOfferLevel_8_2017
+HighOfferLevel_9_2017
+InstSize_1_2017
+InstSize_2_2017
+InstSize_3_2017
+InstSize_4_2017
+InstSize_5_2017
+Location_1_2017
+Location_2_2017
+Location_3_2017
+Location_4_2017
+Sector_1_2017
+Sector_2_2017
+Sector_3_2017
+Sector_4_2017
+Sector_5_2017
+Sector_6_2017
+Sector_7_2017
+Sector_8_2017
+Sector_9_2017
+Calendar_1_2017
+Calendar_2_2017
+Calendar_3_2017
+Calendar_4_2017
+Calendar_5_2017
+Calendar_6_2017
+Calendar_7_2017
+OneYearCert_2017
+TwoYearCert_2017
+AssociateDegree_2017
+FourYearCert_2017
+Bachelor_2017
+PBCert_2017
+Masters_2017
+PMCert_2017
+RelgAffil_2017
+OnCampusHousing_2017
+CarnegieClass_2016
+HighOfferLevel_1_2016
+HighOfferLevel_2_2016
+HighOfferLevel_3_2016
+HighOfferLevel_4_2016
+HighOfferLevel_5_2016
+HighOfferLevel_6_2016
+HighOfferLevel_7_2016
+HighOfferLevel_8_2016
+HighOfferLevel_9_2016
+InstSize_1_2016
+InstSize_2_2016
+InstSize_3_2016
+InstSize_4_2016
+InstSize_5_2016
+Location_1_2016
+Location_2_2016
+Location_3_2016
+Location_4_2016
+Sector_1_2016
+Sector_2_2016
+Sector_3_2016
+Sector_4_2016
+Sector_5_2016
+Sector_6_2016
+Sector_7_2016
+Sector_8_2016
+Sector_9_2016
+Calendar_1_2016
+Calendar_2_2016
+Calendar_3_2016
+Calendar_4_2016
+Calendar_5_2016
+Calendar_6_2016
+Calendar_7_2016
+OneYearCert_2016
+TwoYearCert_2016
+AssociateDegree_2016
+FourYearCert_2016
+Bachelor_2016
+PBCert_2016
+Masters_2016
+PMCert_2016
+RelgAffil_2016
+OnCampusHousing_2016
+CarnegieClass_2015
+HighOfferLevel_1_2015
+HighOfferLevel_2_2015
+HighOfferLevel_3_2015
+HighOfferLevel_4_2015
+HighOfferLevel_5_2015
+HighOfferLevel_6_2015
+HighOfferLevel_7_2015
+HighOfferLevel_8_2015
+HighOfferLevel_9_2015
+InstSize_1_2015
+InstSize_2_2015
+InstSize_3_2015
+InstSize_4_2015
+InstSize_5_2015
+Location_1_2015
+Location_2_2015
+Location_3_2015
+Location_4_2015
+Sector_1_2015
+Sector_2_2015
+Sector_3_2015
+Sector_4_2015
+Sector_5_2015
+Sector_6_2015
+Sector_7_2015
+Sector_8_2015
+Sector_9_2015
+Calendar_1_2015
+Calendar_2_2015
+Calendar_3_2015
+Calendar_4_2015
+Calendar_5_2015
+Calendar_6_2015
+Calendar_7_2015
+OneYearCert_2015
+TwoYearCert_2015
+AssociateDegree_2015
+FourYearCert_2015
+Bachelor_2015
+PBCert_2015
+Masters_2015
+PMCert_2015
+RelgAffil_2015
+OnCampusHousing_2015
+GradRate_2015_dec
+GradRate_2016_dec
+GradRate_2017_dec
+TOTENROLL_2015_log
+TOTENROLL_2016_log